In [1]:
import os
import sys
sys.path.append("..")
import torch
import subprocess
import traceback
import warnings
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

from tqdm import tqdm
# n.pyimport hydra
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

from src.metric import LWLRAP
from src.models import get_model
import src.configuration as C
import src.utils as utils
from src.metric import LWLRAP
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger

/opt/anaconda3/lib/python3.7/site-packages/pywt/_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [16]:
# config
config_filename = 'EfficientNetSED001.yaml'
config = utils.load_config(f"configs/{config_filename}")
device = "cuda"

In [9]:
# data
df, re_df, datadir = C.get_metadata(config)

In [10]:
df

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,data_type
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25,tp
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40,tp
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25,tp
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04,tp
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70,tp
...,...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75,tp
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25,tp
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00,tp
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20,tp


In [13]:
loaders = {
    phase: C.get_loader(df_, datadir, config, phase)
    for df_, phase in zip([df], ["valid"])
}

In [140]:
pseudo_label_df = df[["recording_id"]].copy()

for fold in range(5):
    # model
    print(f"fold-{fold}")
    model = get_model(config)
    model_name = "EfficientNetSED"
    output_dir = Path("output/0119_184426")
    try:
        ckpt = torch.load(output_dir / f'{model_name}-{fold}-v0.ckpt')  # TODO foldごとのモデルを取得できるようにする
    except:
        ckpt = torch.load(output_dir / f'{model_name}-{fold}.ckpt')  # TODO foldごとのモデルを取得できるようにする
    model.load_state_dict(ckpt['state_dict'])
    model.eval().to(device)

    preds = []
    output_key = "logit"
    with torch.no_grad():
        # xは複数のlist
        for x_list, y in tqdm(loaders['valid']):
            batch_size = x_list.shape[0]
            x = x_list.view(-1, x_list.shape[2], x_list.shape[3], x_list.shape[4])  # batch>1でも可
            x = x.to(device)
            output = model.model(x)
            output = output[output_key]
            output = output.view(batch_size, -1, 24)  # 24=num_classes
            pred = torch.max(output, dim=1)[0]  # 1次元目(分割sしたやつ)で各クラスの最大を取得
            pred = torch.argsort(pred, dim=-1, descending=True)
            pred = pred.detach().cpu().numpy()
            preds.append(pred)

    pseudo_label_df[f"fold{fold}"] = np.argmin(np.vstack(preds), axis=1)

fold-0
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:23<00:00,  2.56it/s]
fold-1
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.49it/s]
fold-2
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.50it/s]
fold-3
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.49it/s]
fold-4
Loaded pretrained weights for efficientnet-b2
100%|██████████| 61/61 [00:24<00:00,  2.53it/s]


In [142]:
pseudo_label_df

,recording_id,fold0,fold1,fold2,fold3,fold4
0,003bec244,16,6,19,14,14
1,006ab765f,19,16,19,12,8
2,007f87ba2,7,12,6,7,9
3,0099c367b,21,18,20,19,21
4,009b760e6,12,20,16,11,14
...,...,...,...,...,...,...
1211,fe8d9ac40,13,17,17,11,22
1212,fea6b438a,20,11,23,16,13
1213,ff2eb9ce5,0,0,0,0,0
1214,ffb8d8391,12,18,17,9,5


In [147]:
pseudo_label_list = [] 
for idx, row in pseudo_labels.iterrows():
    recording_id = row
    try:
        pseudo_label = row.value_counts()[(row.value_counts() > 2)].values[0]
    except:
        pseudo_label = np.nan

    pseudo_label_list.append(pseudo_label)


In [148]:
pseudo_label_df["species_id"] = pseudo_label_list

# ひとまずclip単位でラベル付けと言うことにしたいのでtimeは0~60
pseudo_label_df["t_min"] = 0
pseudo_label_df["t_max"] = 60

In [150]:
pseudo_label_df.species_id.value_counts()

3.0    92
5.0    34
4.0    31
Name: species_id, dtype: int64

In [153]:
pseudo_label_df = pseudo_label_df.dropna()

In [154]:
pseudo_label_df = pseudo_label_df[["recording_id", "species_id", "t_min", "t_max"]]

In [157]:
pseudo_label_df["species_id"] = pseudo_label_df["species_id"].astype(int)

In [158]:
pseudo_label_df

,recording_id,species_id,t_min,t_max
6,00d442df7,5,0,60
13,0268057eb,5,0,60
17,02b9a8ab9,3,0,60
21,03d77fede,5,0,60
33,05b9c974c,3,0,60
...,...,...,...,...
1181,f5e6456f0,5,0,60
1186,f7d92975f,3,0,60
1202,fc47cfba3,5,0,60
1203,fc6609050,5,0,60


In [161]:
pseudo_label_df.to_csv("./input/rfcx-species-audio-detection/pseudo.csv", index=False)